## 0) Imports & constants

In [61]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from netCDF4 import Dataset as NC4Dataset
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy import stats
from matplotlib.dates import DateFormatter
from contextlib import redirect_stdout
from pathlib import Path
from datetime import datetime

# ---- Paths (edit to your layout)
cyclone_path = "raw_data/cyclone_data/IBTrACS.last3years.v04r01.nc"
hindcast_dir = "raw_data/wave_hindcast_raw"
buoy_csv     = "raw_data/OTI_wave_buoy/NSWENV_20250121-20250323_20m_OTInth2_WAVE_Parameters.csv"
cds_nc       = "raw_data/CDS/wind_bbox.nc"
STORM        = "ALFRED"  # target storm name (case-insensitive)

# ---- Time window (UTC)
t0 = "2025-02-01"
t1 = "2025-03-31 23:59:59"

# ---- Physical constants
R_EARTH_KM = 6371.0
rho, g = 1025.0, 9.81

## 1) Generic helpers

In [31]:
def pick_var(ds, aliases):
    """Return first existing variable name from a list of aliases."""
    for name in aliases:
        if name in ds:
            return name
    return None

def haversine_km(lat1, lon1, lat2, lon2):
    """Scalar haversine distance in km."""
    la1, lo1 = np.radians(lat1), np.radians(lon1)
    la2, lo2 = np.radians(lat2), np.radians(lon2)
    dlat, dlon = la2 - la1, lo2 - lo1
    a = np.sin(dlat/2.0)**2 + np.cos(la1)*np.cos(la2)*np.sin(dlon/2.0)**2
    return 2 * R_EARTH_KM * np.arcsin(np.sqrt(a))

def initial_bearing_deg(lat1, lon1, lat2, lon2):
    """Scalar initial bearing (deg, 0..360)."""
    la1, lo1 = np.radians(lat1), np.radians(lon1)
    la2, lo2 = np.radians(lat2), np.radians(lon2)
    dlon = lo2 - lo1
    y = np.sin(dlon) * np.cos(la2)
    x = np.cos(la1)*np.sin(la2) - np.sin(la1)*np.cos(la2)*np.cos(dlon)
    return (np.degrees(np.arctan2(y, x)) + 360.0) % 360.0

def _haversine_nan(lat0, lon0, lat_arr, lon_arr):
    """Vectorized haversine vs a fixed point, respects NaNs."""
    la = np.radians(lat_arr.astype(float))
    lo = np.radians(lon_arr.astype(float))
    la0, lo0 = np.radians(lat0), np.radians(lon0)
    dlat, dlon = la - la0, lo - lo0
    a = np.sin(dlat/2.0)**2 + np.cos(la0)*np.cos(la)*np.sin(dlon/2.0)**2
    return 2 * R_EARTH_KM * np.arcsin(np.sqrt(a))

def _bearing_nan(lat0, lon0, lat_arr, lon_arr):
    """Vectorized initial bearing vs a fixed point, respects NaNs."""
    la = np.radians(lat_arr.astype(float))
    lo = np.radians(lon_arr.astype(float))
    la0, lo0 = np.radians(lat0), np.radians(lon0)
    dlon = lo - lo0
    y = np.sin(dlon) * np.cos(la)
    x = np.cos(la0)*np.sin(la) - np.sin(la0)*np.cos(la)*np.cos(dlon)
    return (np.degrees(np.arctan2(y, x)) + 360.0) % 360.0

def norm_lon_180(lon):
    """Normalize longitudes to [-180, 180)."""
    return ((np.asarray(lon, dtype='float64') + 180.0) % 360.0) - 180.0

def circ_mean_deg(a):
    """Circular mean (deg) with NaN handling."""
    a = np.asarray(a, dtype='float64')
    a = a[~np.isnan(a)]
    if len(a) == 0: return np.nan
    rad = np.deg2rad(a)
    return (np.degrees(np.arctan2(np.nanmean(np.sin(rad)), np.nanmean(np.cos(rad)))) + 360.0) % 360.0

def circ_diff(a, b):
    """Smallest signed angular difference b–a (deg, −180..180)."""
    if np.isnan(a) or np.isnan(b): return np.nan
    return ((b - a + 540) % 360) - 180

def _bytes_to_str(a):
    """Decode bytes to str; leave str as-is."""
    if isinstance(a, (bytes, bytearray)):
        return a.decode('utf-8', 'ignore')
    return str(a)

def _match_storm_mask(name_da, target):
    """Case-insensitive exact storm name match (handles bytes)."""
    target_up = str(target).strip().upper()
    v = np.array([_bytes_to_str(x).strip().upper() for x in name_da.values])
    return v == target_up

def _extract_time(st):
    """Robust time extraction from IBTrACS slice."""
    if 'time' in st.coords:
        return pd.to_datetime(st['time'].values)
    for cand in ['iso_time','ISO_TIME']:
        if cand in st.variables:
            arr = st[cand].astype(str).values
            arr = np.where(pd.Series(arr).str.strip().astype(bool), arr, np.nan)
            return pd.to_datetime(arr, errors='coerce')
    for cand in ['date_time','DATE_TIME']:
        if cand in st.variables:
            try:
                return pd.to_datetime(xr.decode_cf(st[[cand]])[cand].values)
            except Exception:
                dt = st[cand]
                units = dt.attrs.get('units', 'days since 1858-11-17 00:00:00')
                base  = pd.Timestamp(units.split('since',1)[1].strip()) if 'since' in units else pd.Timestamp('1858-11-17')
                unit0 = units.split()[0].lower() if ' ' in units else 'days'
                unit  = 'D' if unit0.startswith('day') else ('h' if unit0.startswith('hour') else 's')
                return base + pd.to_timedelta(dt.values, unit=unit)
    raise ValueError("No usable time in IBTrACS (looked for: time, iso_time, date_time).")

def _clean_indexed(df):
    """Ensure datetime index, uniqueness, sorted; combine duplicates by mean."""
    s = df.copy()
    s.index = pd.to_datetime(s.index, errors='coerce')
    s = s[~s.index.isna()].sort_index()
    if not s.index.is_unique:
        s = s.groupby(level=0).mean(numeric_only=True)
    return s


# --- Global figure saving setup ---
SAVE_FIGS = True               # toggle ON/OFF
FIG_DIR   = "figures_out"      # where to save
os.makedirs(FIG_DIR, exist_ok=True)

def save_all_open_figs(prefix="fig", fmt="png", close=True):
    """Save all currently open matplotlib figures to disk."""
    for i, num in enumerate(plt.get_fignums(), 1):
        fig = plt.figure(num)
        fname = f"{prefix}_{i:02d}.{fmt}"
        path = os.path.join(FIG_DIR, fname)
        fig.savefig(path, dpi=300, bbox_inches="tight")
        print(f"Saved: {path}")
        if close:
            plt.close(fig)


## 2) Load & harmonise hindcast (waves)

In [4]:
# Discover valid .nc files under hindcast_dir
raw_files = [os.path.join(root, f)
             for root, _, files in os.walk(hindcast_dir)
             for f in files if f.endswith(".nc")]

valid_files = []
for f in raw_files:  # Skip corrupted netCDFs
    try:
        with NC4Dataset(f) as _:
            pass
        valid_files.append(f)
    except OSError:
        print(f"Skipping broken file: {f}")

if not valid_files:
    raise FileNotFoundError("No valid hindcast .nc files found.")

# Open all and concat along a synthetic 'file' dim
datasets = [xr.open_dataset(f, decode_times=True) for f in valid_files]
ds_wave = xr.concat(datasets, dim="file")

# Normalize coord names to lat/lon
rename_map = {}
if 'longitude' in ds_wave.coords and 'lon' not in ds_wave.coords: rename_map['longitude'] = 'lon'
if 'latitude'  in ds_wave.coords and 'lat' not in ds_wave.coords:  rename_map['latitude']  = 'lat'
ds_wave = ds_wave.rename(rename_map)

# Core vars via aliases
hs_name  = pick_var(ds_wave, ['hs','Hm0','Hsig','Hm0_wave','significant_wave_height'])
tp_name  = pick_var(ds_wave, ['tp','Tp','tpeak','t02','Tm02'])
fp_name  = pick_var(ds_wave, ['fp','Fp','fpeak'])
dir_name = pick_var(ds_wave, ['dir','Dp','dp','direction'])
cg_name  = pick_var(ds_wave, ['cge','cg','C_g','group_velocity'])

core_vars = [v for v in [hs_name, tp_name, fp_name, dir_name, cg_name] if v is not None]
if not core_vars: raise ValueError("No wave variables found (hs/tp/fp/dir/cg).")
if 'time' not in ds_wave.coords: raise ValueError("Hindcast dataset has no 'time' coordinate.")

# Time slice and -> tidy DataFrame
ds_wave = ds_wave.sel(time=slice(t0, t1))[core_vars]
df_wave = (ds_wave.to_dataframe().reset_index().set_index('time').sort_index())

# Harmonize variable names for downstream
rename_cols = {}
if hs_name:  rename_cols[hs_name]  = 'hs'
if tp_name:  rename_cols[tp_name]  = 'tp' if 'tp' in tp_name.lower() else 't02'
if fp_name:  rename_cols[fp_name]  = 'fp'
if dir_name: rename_cols[dir_name] = 'dp'
if cg_name:  rename_cols[cg_name]  = 'cge'
df_wave = df_wave.rename(columns=rename_cols)

Skipping broken file: raw_data/wave_hindcast_raw\waves_point_444428.0_lon152.267_lat-23.733_202503.nc


## 3) Load & harmonise ERA5 (wind/pressure)

In [5]:
ds_cds = xr.open_dataset(cds_nc, decode_times=True)

coord_renames = {}
if 'valid_time' in ds_cds.coords and 'time' not in ds_cds.coords: coord_renames['valid_time'] = 'time'
if 'latitude'   in ds_cds.coords and 'lat' not in ds_cds.coords:   coord_renames['latitude']   = 'lat'
if 'longitude'  in ds_cds.coords and 'lon' not in ds_cds.coords:   coord_renames['longitude']  = 'lon'
ds_cds = ds_cds.rename(coord_renames)

u_name  = pick_var(ds_cds, ['u10','10u','u'])
v_name  = pick_var(ds_cds, ['v10','10v','v'])
sp_name = pick_var(ds_cds, ['sp','surface_pressure'])
need = [u_name, v_name, sp_name]
if any(v is None for v in need):
    raise ValueError(f"Missing ERA5 vars. Found u={u_name}, v={v_name}, sp={sp_name}")
if 'time' not in ds_cds.coords:
    raise ValueError("ERA5 dataset has no 'time' coord.")

# Trim and clean admin dims/coords
ds_cds = ds_cds.sel(time=slice(t0, t1))[need]
for maybe_admin in ['expver','number','step','surface']:
    if maybe_admin in ds_cds.coords and maybe_admin not in ds_cds.dims:
        ds_cds = ds_cds.drop_vars(maybe_admin)
    if maybe_admin in ds_cds.dims and ds_cds.dims[maybe_admin] == 1:
        ds_cds = ds_cds.isel({maybe_admin: 0})

df_cds = (ds_cds.to_dataframe().reset_index().set_index('time').sort_index()
          .rename(columns={u_name:'u10', v_name:'v10', sp_name:'sp_pa'}))
df_cds['sp_hpa']  = df_cds['sp_pa'] / 100.0
df_cds['wind_ms'] = np.sqrt(df_cds['u10']**2 + df_cds['v10']**2)

## 4) Load IBTrACS & isolate target storm

In [6]:
ds_cyc = xr.open_dataset(cyclone_path, decode_times=True)
if 'name' not in ds_cyc: raise ValueError("IBTrACS lacks 'name' variable.")

mask = _match_storm_mask(ds_cyc['name'], STORM)
if not mask.any(): raise ValueError(f"Storm '{STORM}' not found in IBTrACS.")
st = ds_cyc.sel(storm=mask).squeeze(drop=True)

lat_var = 'lat' if 'lat' in st.variables else ('latitude' if 'latitude' in st.variables else None)
lon_var = 'lon' if 'lon' in st.variables else ('longitude' if 'longitude' in st.variables else None)
if not lat_var or not lon_var:
    raise ValueError(f"IBTrACS lat/lon not found. Have: {list(st.variables)}")

lat_raw  = st[lat_var].values.astype('float64')
lon_raw  = st[lon_var].values.astype('float64')
time_raw = _extract_time(st)

valid = np.isfinite(lat_raw) & np.isfinite(lon_raw) & pd.notna(time_raw)
df_cyc = (pd.DataFrame({
    'time':   pd.to_datetime(np.asarray(time_raw)[valid]),
    'cyc_lat': lat_raw[valid],
    'cyc_lon': norm_lon_180(lon_raw[valid]),
}).sort_values('time')
  .loc[lambda d: (d['time'] >= pd.Timestamp(t0)) & (d['time'] <= pd.Timestamp(t1))]
  .reset_index(drop=True)
)
print(f"IBTrACS rows for {STORM} in window:", len(df_cyc))

# Hourly cyclone window for masking outside true existence
df_cyc_hr_native = (df_cyc.set_index('time').resample('1H').mean()
                    .interpolate('time', limit_direction='both'))
t_first = df_cyc_hr_native.index.min()
t_last  = df_cyc_hr_native.index.max()
print("Cyclone native window:", t_first, "→", t_last)

IBTrACS rows for ALFRED in window: 129
Cyclone native window: 2025-02-21 00:00:00 → 2025-03-08 18:00:00


C:\Users\arc\miniconda3\envs\envireef_core\lib\site-packages\xarray\coding\times.py:242: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
C:\Users\arc\miniconda3\envs\envireef_core\lib\site-packages\xarray\coding\times.py:242: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


## 5) Sector geometry

In [7]:
SECTOR_BANDS = {
    'North':   {'lat_min': -22.97764333, 'lat_max': -22.46652985},
    'Central': {'lat_min': -23.48875682, 'lat_max': -22.97764333},
    'South':   {'lat_min': -23.9998703,  'lat_max': -23.48875682},
}
# Swapped lon centers accordingly
SECTOR_LON_CENTERS = {'North': 152.30, 'Central': 152.15, 'South': 152.05}

def _mid_lat(band): return 0.5*(band['lat_min'] + band['lat_max'])

sector_centroids = pd.DataFrame(
    [{'sector': name, 'sector_lat': _mid_lat(band), 'sector_lon': SECTOR_LON_CENTERS[name]}
     for name, band in SECTOR_BANDS.items()]
)

SECTOR_BOUNDS = {
    k: {'lat_min': v['lat_min'], 'lat_max': v['lat_max'], 'lon_min': 151.70, 'lon_max': 152.50}
    for k, v in SECTOR_BANDS.items()
}
print(sector_centroids)

    sector  sector_lat  sector_lon
0    North  -22.722087      152.30
1  Central  -23.233200      152.15
2    South  -23.744314      152.05


## 6) Build master hourly index & aligned series

In [11]:
# Wave → average across grid: one series per timestamp
wave_cols = [c for c in ['hs','tp','t02','dp','cge','fp'] if c in df_wave.columns]
df_wave_ts = (df_wave[wave_cols].groupby(level='time').mean(numeric_only=True).sort_index())
df_wave_ts['tp_use'] = df_wave_ts['tp'] if 'tp' in df_wave_ts.columns else df_wave_ts.get('t02', np.nan)

# ERA5 → average across bbox to one series
cds_cols = [c for c in ['u10','v10','sp_pa','sp_hpa','wind_ms'] if c in df_cds.columns]
df_cds_ts = (df_cds[cds_cols].groupby(level='time').mean(numeric_only=True).sort_index())
df_cds_ts['pressure_anom'] = 1013.25 - (df_cds_ts['sp_hpa'] if 'sp_hpa' in df_cds_ts.columns else df_cds_ts['sp_pa']/100.0)

# Clean indices
df_wave_ts = _clean_indexed(df_wave_ts)
df_cds_ts  = _clean_indexed(df_cds_ts)
df_cyc_ix  = df_cyc.copy()
df_cyc_ix['time'] = pd.to_datetime(df_cyc_ix['time'], errors='coerce')
df_cyc_ix = df_cyc_ix.dropna(subset=['time']).sort_values('time').set_index('time')

# Master hourly index for [t0..t1]
master_index = pd.date_range(start=pd.Timestamp(t0).floor('H'),
                             end=pd.Timestamp(t1).ceil('H'), freq='1H')

# Align to master
wave_hourly = (df_wave_ts.resample('1H').mean().reindex(master_index).interpolate('time', limit=6))
cds_hourly  = (df_cds_ts.resample('1H').mean().reindex(master_index).interpolate('time', limit=6))
cyc_hourly_native = (df_cyc_ix[['cyc_lat','cyc_lon']].resample('1H').mean()
                     .interpolate('time', limit_direction='both'))
cyc_hourly = cyc_hourly_native.reindex(master_index)
outside = (cyc_hourly.index < t_first) | (cyc_hourly.index > t_last)
cyc_hourly.loc[outside, ['cyc_lat','cyc_lon']] = np.nan

# Combined aligned (before sectors)
aligned = (wave_hourly.join(cds_hourly, how='left', rsuffix='_cds')
           .join(cyc_hourly, how='left'))

# Ensure derived forcings exist
if 'wind_ms' not in aligned and {'u10','v10'} <= set(aligned.columns):
    aligned['wind_ms'] = np.sqrt(aligned['u10']**2 + aligned['v10']**2)
if 'pressure_anom' not in aligned:
    if 'sp_hpa' in aligned:
        aligned['pressure_anom'] = 1013.25 - aligned['sp_hpa']
    elif 'sp_pa' in aligned:
        aligned['pressure_anom'] = 1013.25 - (aligned['sp_pa'] / 100.0)

# Reusable masks
obs_start, obs_end = pd.Timestamp(t0), pd.Timestamp(t1)
in_obs  = (aligned.index >= obs_start) & (aligned.index <= obs_end)
in_evt  = (aligned.index >= t_first)   & (aligned.index <= t_last)
pre_evt  = (aligned.index >= obs_start) & (aligned.index <  t_first)
post_evt = (aligned.index >  t_last)    & (aligned.index <= obs_end)

print("Windows:")
print(" Wave:   ", wave_hourly.index.min(), "→", wave_hourly.index.max(), "| rows:", len(wave_hourly))
print(" ERA5:   ", cds_hourly.index.min(),  "→", cds_hourly.index.max(),  "| rows:", len(cds_hourly))
print(" Cyclone:", t_first, "→", t_last,                            "| rows:", len(cyc_hourly_native))
print(" Master: ", master_index.min(), "→", master_index.max(),     "| rows:", len(master_index))

Windows:
 Wave:    2025-02-01 00:00:00 → 2025-04-01 00:00:00 | rows: 1417
 ERA5:    2025-02-01 00:00:00 → 2025-04-01 00:00:00 | rows: 1417
 Cyclone: 2025-02-21 00:00:00 → 2025-03-08 18:00:00 | rows: 379
 Master:  2025-02-01 00:00:00 → 2025-04-01 00:00:00 | rows: 1417


## 9) Sector time series builder and regressions

In [12]:
def _sector_wave_timeseries(ds_wave, bounds, t0, t1, master_index, rename_cols):
    """Subset hindcast to bounds; hourly ts with harmonized names + tp_use."""
    latb = (ds_wave['lat'] >= bounds['lat_min']) & (ds_wave['lat'] <= bounds['lat_max'])
    lonb = (ds_wave['lon'] >= bounds['lon_min']) & (ds_wave['lon'] <= bounds['lon_max'])
    mask = (latb & lonb)
    if mask.sum().item() == 0:
        raise ValueError(f"No hindcast grid points inside bounds: {bounds}")
    ds_sec = ds_wave.sel(file=mask).sel(time=slice(t0, t1))
    df = (ds_sec.to_dataframe().reset_index().set_index('time').sort_index()).rename(columns=rename_cols)
    wave_cols = [c for c in ['hs','tp','t02','dp','cge','fp'] if c in df.columns]
    if not wave_cols:
        raise ValueError("Sector subset has no wave variables after renaming.")
    df_ts = (df[wave_cols].groupby(level='time').mean(numeric_only=True).sort_index())
    df_ts['tp_use'] = df_ts['tp'] if 'tp' in df_ts.columns else df_ts.get('t02', np.nan)
    return (df_ts.resample('1H').mean().reindex(master_index).interpolate('time', limit=6))

# Build sectors with distance/bearing/energy flux
sector_frames = {}
for _, srow in sector_centroids.iterrows():
    sector = srow['sector']
    lat0, lon0 = float(srow['sector_lat']), float(srow['sector_lon'])
    bounds = SECTOR_BOUNDS[sector]

    wave_sec = _sector_wave_timeseries(ds_wave, bounds, t0, t1, master_index, rename_cols)
    ts = (wave_sec
          .join(cds_hourly[['wind_ms','sp_hpa']] if 'sp_hpa' in cds_hourly.columns else cds_hourly[['wind_ms','sp_pa']], how='left')
          .join(cyc_hourly[['cyc_lat','cyc_lon']], how='left'))

    if 'sp_hpa' not in ts and 'sp_pa' in ts:
        ts['sp_hpa'] = ts['sp_pa'] / 100.0
    ts['pressure_anom'] = 1013.25 - ts['sp_hpa']

    ts['dist_km'] = _haversine_nan(lat0, lon0, ts['cyc_lat'].values, ts['cyc_lon'].values)
    ts['bearing'] = _bearing_nan(lat0, lon0, ts['cyc_lat'].values, ts['cyc_lon'].values)

    ts['cge_use'] = ts['cge'] if 'cge' in ts.columns else (g/(4.0*np.pi)) * ts['tp_use']
    if 'hs' in ts.columns:
        ts['energy_flux'] = 0.5 * rho * g * (ts['hs']**2) * ts['cge_use']

    keep = ['hs','tp_use','dp','wind_ms','pressure_anom','dist_km','bearing','energy_flux']
    sector_frames[sector] = ts[[c for c in keep if c in ts.columns]]

print("\nSector wave sanity check (hs means):")
for s, ts in sector_frames.items():
    print(f"  {s:7s}: hs mean={ts['hs'].mean():.3f}  dp mean={ts['dp'].mean():.1f}")


Sector wave sanity check (hs means):
  North  : hs mean=2.087  dp mean=103.2
  Central: hs mean=2.000  dp mean=99.0
  South  : hs mean=1.730  dp mean=91.5


In [13]:
print("\n=== Directional Shift Analysis (per sector) ===")
rows_dir = []
for sector, ts in sector_frames.items():
    pre  = ts.loc[pre_evt,  'dp'].dropna() if 'dp' in ts else pd.Series(dtype=float)
    peak = ts.loc[in_evt,   'dp'].dropna() if 'dp' in ts else pd.Series(dtype=float)
    post = ts.loc[post_evt, 'dp'].dropna() if 'dp' in ts else pd.Series(dtype=float)
    pre_m, peak_m, post_m = circ_mean_deg(pre), circ_mean_deg(peak), circ_mean_deg(post)
    d_pre_peak  = circ_diff(pre_m,  peak_m)
    d_peak_post = circ_diff(peak_m, post_m)
    d_pre_post  = circ_diff(pre_m,  post_m)
    print(f"{sector:8s}  mean°  pre:{pre_m:6.1f}  peak:{peak_m:6.1f}  post:{post_m:6.1f}  "
          f"Δ(pre→peak):{d_pre_peak:6.1f}  Δ(peak→post):{d_peak_post:6.1f}  Δ(pre→post):{d_pre_post:6.1f}")
    rows_dir.append({'sector': sector, 'pre_mean': pre_m, 'peak_mean': peak_m, 'post_mean': post_m,
                     'Δ_pre_peak': d_pre_peak, 'Δ_peak_post': d_peak_post, 'Δ_pre_post': d_pre_post})
df_dirshift = pd.DataFrame(rows_dir)


=== Directional Shift Analysis (per sector) ===
North     mean°  pre: 104.8  peak: 114.0  post:  96.3  Δ(pre→peak):   9.1  Δ(peak→post): -17.7  Δ(pre→post):  -8.6
Central   mean°  pre: 101.1  peak: 107.5  post:  92.3  Δ(pre→peak):   6.4  Δ(peak→post): -15.2  Δ(pre→post):  -8.9
South     mean°  pre:  94.7  peak:  96.6  post:  86.2  Δ(pre→peak):   1.9  Δ(peak→post): -10.4  Δ(pre→post):  -8.5


In [14]:
def _ols_report(name, y, X):
    """Fit OLS, print compact summary table."""
    Xc = sm.add_constant(X, has_constant='add')
    m  = sm.OLS(y, Xc, missing='drop').fit()
    print(f"{name}: R²={m.rsquared:.3f} | n={int(m.nobs)}")
    print(m.summary().tables[1])
    return m

def regress_hs_distance(ts):
    """Hs ~ dist_km (+ optional quadratic) during event."""
    d = ts.loc[in_evt, ['hs','dist_km']].dropna()
    X = pd.DataFrame({'dist_km': d['dist_km'], 'dist_km2': d['dist_km']**2})
    return sm.OLS(d['hs'], sm.add_constant(X)).fit()

def regress_exposure(ts):
    """Hs ~ wind + dist + pressure during event."""
    cols = ['hs','wind_ms','pressure_anom','dist_km']
    d = ts.loc[in_evt, cols].dropna()
    return sm.OLS(d['hs'], sm.add_constant(d[['wind_ms','pressure_anom','dist_km']])).fit()

# Sector regressions
for sector, ts in sector_frames.items():
    m1 = regress_hs_distance(ts)
    m2 = regress_exposure(ts)
    print(f"\n[{sector}] Hs~dist: R2={m1.rsquared:.3f}, n={int(m1.nobs)}")
    print(f"[{sector}] Hs~wind+dist+press: R2={m2.rsquared:.3f}, n={int(m2.nobs)}")

# Buoy regressions
df_reg = oti.loc[peak_mask, ['hs','dist_km','wind_ms','pressure_anom']].dropna()
m_dist     = _ols_report("Hs ~ dist",               df_reg['hs'], df_reg[['dist_km']])
m_wind     = _ols_report("Hs ~ wind",               df_reg['hs'], df_reg[['wind_ms']])
m_press    = _ols_report("Hs ~ press",              df_reg['hs'], df_reg[['pressure_anom']])
m_exposure = _ols_report("Hs ~ wind+dist+press",    df_reg['hs'], df_reg[['wind_ms','dist_km','pressure_anom']])
df_reg_poly = df_reg.assign(dist_km2=lambda d: d['dist_km']**2)
m_dist2     = _ols_report("Hs ~ dist + dist²",      df_reg_poly['hs'], df_reg_poly[['dist_km','dist_km2']])

# Single-var quick scan per sector
def single_var_reg(ts, xvar):
    cols = ['hs', xvar]
    d = ts.loc[in_evt, cols].dropna()
    if len(d) < 10: return None
    m = sm.OLS(d['hs'], sm.add_constant(d[xvar])).fit()
    return xvar, m.rsquared, len(d), m.params.get(xvar, np.nan), m.pvalues.get(xvar, np.nan)

for sector, ts in sector_frames.items():
    results = [single_var_reg(ts, var) for var in ['wind_ms', 'pressure_anom', 'dp']]
    results = [r for r in results if r]
    print(f"\n{sector} — Hs vs single forcings (cyclone window)")
    for var, r2, n, coef, p in results:
        print(f"  {var:>14} | R²={r2:5.3f} | n={n:3d} | coef={coef:8.3f} | p={p:8.3e}")

# Example access (kept as in your code)
model = m_exposure
print(model.pvalues)

# Energy flux stats per sector (±3 d)
win = (aligned.index >= (t_first - pad)) & (aligned.index <= (t_last + pad))
for sector, ts in sector_frames.items():
    sub = ts.loc[win]
    if 'energy_flux' in sub:
        print(f"{sector}: mean P={sub['energy_flux'].mean():.1f}, max P={sub['energy_flux'].max():.1f}")


[North] Hs~dist: R2=0.371, n=379
[North] Hs~wind+dist+press: R2=0.661, n=379

[Central] Hs~dist: R2=0.187, n=379
[Central] Hs~wind+dist+press: R2=0.685, n=379

[South] Hs~dist: R2=0.054, n=379
[South] Hs~wind+dist+press: R2=0.756, n=379
Hs ~ dist: R²=0.053 | n=379
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8850      0.118     15.916      0.000       1.652       2.118
dist_km       -0.0007      0.000     -4.577      0.000      -0.001      -0.000
Hs ~ wind: R²=0.277 | n=379
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3339      0.093      3.588      0.000       0.151       0.517
wind_ms        0.1058      0.009     12.021      0.000       0.088       0.123
Hs ~ press: R²=0.039 | n=379
                    coef    std err          

## 8) Load OTI buoy, build hourly series and regressions

In [15]:
def _maybe_parse_datetime(col):
    try:
        return pd.to_datetime(col, errors='raise')
    except Exception:
        return pd.to_datetime(col, errors='coerce')

df_buoy_raw = pd.read_csv(buoy_csv)
time_candidates = [c for c in df_buoy_raw.columns if 'time' in c.lower() or 'date' in c.lower()]
if not time_candidates:
    raise ValueError("Buoy CSV missing a datetime column.")
time_col_buoy = time_candidates[0]

df_buoy = df_buoy_raw.copy()
df_buoy[time_col_buoy] = _maybe_parse_datetime(df_buoy[time_col_buoy])
df_buoy = (df_buoy.loc[(df_buoy[time_col_buoy] >= pd.Timestamp(t0)) & (df_buoy[time_col_buoy] <= pd.Timestamp(t1))]
                  .set_index(time_col_buoy).sort_index())

# Rename likely columns → minimal set
rename_buoy = {'Hm0':'hs','Hsig':'hs','Tp':'tp','Tz':'tz','Tm02':'t02','Dp':'dp','Dm':'dm','Dp_sp':'dp_sp','Dm_sp':'dm_sp','temp':'temp'}
df_buoy = df_buoy.rename(columns={k:v for k,v in rename_buoy.items() if k in df_buoy.columns})
df_buoy = df_buoy.loc[:, ~df_buoy.columns.duplicated()].copy()

# Fill aliases if needed
alias = {'Hs':'hs','Hm0':'hs','Hsig':'hs','Tp':'tp','Tz':'tz','Tm02':'t02','Dp':'dp','Dm':'dm','Dp_sp':'dp_sp','Dm_sp':'dm_sp','Temp':'temp'}
for k,v in alias.items():
    if k in df_buoy.columns and v not in df_buoy.columns:
        df_buoy.rename(columns={k:v}, inplace=True)

# OTI hourly series
keep_buoy = [c for c in ['hs','tp','t02','dp','tz','temp'] if c in df_buoy.columns]
buoy_hourly = (df_buoy[keep_buoy].resample('1H').mean(numeric_only=True)
               .reindex(pd.date_range(pd.Timestamp(t0).floor('H'), pd.Timestamp(t1).ceil('H'), freq='1H'))
               .interpolate('time', limit=4))
buoy_hourly['tp_use'] = buoy_hourly['tp'] if 'tp' in buoy_hourly.columns else buoy_hourly.get('t02', np.nan)

# Join forcings + cyclone; compute distance/bearing from OTI
OTI = {'name': 'One Tree Island', 'lat': -23.504, 'lon': 152.094}
oti = (buoy_hourly
       .join(cds_hourly[['wind_ms','sp_hpa']] if 'sp_hpa' in cds_hourly.columns else cds_hourly[['wind_ms','sp_pa']], how='left')
       .join(cyc_hourly[['cyc_lat','cyc_lon']], how='left'))
if 'sp_hpa' not in oti and 'sp_pa' in oti:
    oti['sp_hpa'] = oti['sp_pa']/100.0
oti['pressure_anom'] = 1013.25 - oti['sp_hpa']
oti['dist_km'] = _haversine_nan(OTI['lat'], OTI['lon'], oti['cyc_lat'].values, oti['cyc_lon'].values)
oti['bearing'] = _bearing_nan(OTI['lat'], OTI['lon'], oti['cyc_lat'].values, oti['cyc_lon'].values)
oti['cge_est'] = (g/(4.0*np.pi)) * oti['tp_use']
oti['P_buoy']  = 0.5 * rho * g * (oti['hs']**2) * oti['cge_est']

# Phase masks for buoy
pad = pd.Timedelta(days=3)
pre_mask  = (oti.index >= pd.Timestamp(t0)) & (oti.index <  t_first)
peak_mask = (oti.index >= t_first) & (oti.index <= t_last)
post_mask = (oti.index >  t_last) & (oti.index <= pd.Timestamp(t1))
win_mask  = (oti.index >= (t_first - pad)) & (oti.index <= (t_last + pad))

print("df_buoy shape:", df_buoy.shape)

df_buoy shape: (2413, 18)


In [16]:
def _oti_buoy_timeseries(buoy_hourly, master_index, cds_hourly, cyc_hourly, OTI_lat, OTI_lon):
    """
    OTI buoy -> hourly ts aligned to master_index with the same columns as sector frames:
    ['hs','tp_use','dp','wind_ms','pressure_anom','dist_km','bearing','energy_flux']
    """
    # 1) Start from buoy_hourly; ensure index is hourly and aligned
    bu = (buoy_hourly
          .reindex(master_index)                # align to master clock
          .interpolate('time', limit=4))        # small gaps fill

    # 2) tp_use (prefer Tp else Tm02)
    if 'tp_use' not in bu.columns:
        bu['tp_use'] = bu['tp'] if 'tp' in bu.columns else bu.get('t02', np.nan)

    # 3) Join ERA5 wind/pressure + cyclone center
    ts = (bu
          .join(cds_hourly[['wind_ms','sp_hpa']] if 'sp_hpa' in cds_hourly.columns
                else cds_hourly[['wind_ms','sp_pa']], how='left')
          .join(cyc_hourly[['cyc_lat','cyc_lon']], how='left'))

    # 4) Pressure anomaly (hPa)
    if 'sp_hpa' not in ts and 'sp_pa' in ts:
        ts['sp_hpa'] = ts['sp_pa'] / 100.0
    ts['pressure_anom'] = 1013.25 - ts['sp_hpa']

    # 5) Distance & bearing from OTI to cyclone
    ts['dist_km'] = _haversine_nan(OTI_lat, OTI_lon, ts['cyc_lat'].values, ts['cyc_lon'].values)
    ts['bearing'] = _bearing_nan(OTI_lat, OTI_lon, ts['cyc_lat'].values, ts['cyc_lon'].values)

    # 6) Energy flux (deep-water Cg ≈ g/(4π)·T)
    ts['cge_use'] = (g/(4.0*np.pi)) * ts['tp_use']
    if 'hs' in ts.columns:
        ts['energy_flux'] = 0.5 * rho * g * (ts['hs']**2) * ts['cge_use']

    # 7) Keep same column set as sectors
    keep = ['hs','tp_use','dp','wind_ms','pressure_anom','dist_km','bearing','energy_flux']
    return ts[[c for c in keep if c in ts.columns]]

# Build OTI frame
OTI = {'name': 'One Tree Island', 'lat': -23.504, 'lon': 152.094}  # if not already defined
oti_frame = _oti_buoy_timeseries(buoy_hourly, master_index, cds_hourly, cyc_hourly,
                                 OTI_lat=OTI['lat'], OTI_lon=OTI['lon'])

# (optional) register as another "sector" so downstream loops include OTI
sector_frames['OTI'] = oti_frame

# Quick sanity check (same as sectors)
print("\nOTI buoy sanity check:")
print(f"  OTI    : hs mean={oti_frame['hs'].mean():.3f}  dp mean={oti_frame['dp'].mean():.1f}")



OTI buoy sanity check:
  OTI    : hs mean=1.120  dp mean=55.2


In [17]:
# === Directional Shift Analysis — OTI buoy ====================================
print("\n=== Directional Shift Analysis (OTI buoy) ===")

if 'dp' not in oti.columns:
    print("[OTI] No 'dp' column found in buoy data — skipping.")
    df_dirshift_oti = pd.DataFrame(columns=['series','pre_mean','peak_mean','post_mean',
                                            'Δ_pre_peak','Δ_peak_post','Δ_pre_post','n_pre','n_peak','n_post'])
else:
    pre_dp  = oti.loc[pre_mask,  'dp'].dropna()
    peak_dp = oti.loc[peak_mask, 'dp'].dropna()
    post_dp = oti.loc[post_mask, 'dp'].dropna()

    pre_m  = circ_mean_deg(pre_dp.values)
    peak_m = circ_mean_deg(peak_dp.values)
    post_m = circ_mean_deg(post_dp.values)

    d_pre_peak  = circ_diff(pre_m,  peak_m)
    d_peak_post = circ_diff(peak_m, post_m)
    d_pre_post  = circ_diff(pre_m,  post_m)

    print(f"OTI      mean°  pre:{pre_m:6.1f}  peak:{peak_m:6.1f}  post:{post_m:6.1f}  "
          f"Δ(pre→peak):{d_pre_peak:6.1f}  Δ(peak→post):{d_peak_post:6.1f}  Δ(pre→post):{d_pre_post:6.1f}")
    print(f"         n      pre:{len(pre_dp):6d}  peak:{len(peak_dp):6d}  post:{len(post_dp):6d}")

    # tidy summary frame (mirrors your sector df)
    df_dirshift_oti = pd.DataFrame([{
        'series': 'OTI buoy',
        'pre_mean': pre_m, 'peak_mean': peak_m, 'post_mean': post_m,
        'Δ_pre_peak': d_pre_peak, 'Δ_peak_post': d_peak_post, 'Δ_pre_post': d_pre_post,
        'n_pre': len(pre_dp), 'n_peak': len(peak_dp), 'n_post': len(post_dp)
    }])

# optional: display rounded table
try:
    display(df_dirshift_oti.round(1))
except Exception:
    print("\nOTI directional shift summary:")
    print(df_dirshift_oti.round(1))



=== Directional Shift Analysis (OTI buoy) ===
OTI      mean°  pre:  52.3  peak:  48.4  post:  51.6  Δ(pre→peak):  -3.9  Δ(peak→post):   3.1  Δ(pre→post):  -0.8
         n      pre:   480  peak:   379  post:   352


,series,pre_mean,peak_mean,post_mean,Δ_pre_peak,Δ_peak_post,Δ_pre_post,n_pre,n_peak,n_post
0,OTI buoy,52.3,48.4,51.6,-3.9,3.1,-0.8,480,379,352


In [54]:
def _ols_report(name, y, X):
    """Fit OLS, print compact summary table (with full p-value precision)."""
    Xc = sm.add_constant(X, has_constant='add')
    m  = sm.OLS(y, Xc, missing='drop').fit()
    print(f"{name}: R²={m.rsquared:.3f} | n={int(m.nobs)}")

    # Rebuild our own table so p-values keep full precision
    print("==============================================================================")
    print(f"{'':20s} {'coef':>12s} {'std err':>12s} {'t':>10s} {'P>|t|':>14s} {'[0.025':>12s} {'0.975]':>12s}")
    print("------------------------------------------------------------------------------")
    for param in m.params.index:
        coef = m.params[param]
        se   = m.bse[param]
        tval = m.tvalues[param]
        pval = m.pvalues[param]
        ci_low, ci_high = m.conf_int().loc[param]
        print(f"{param:20s} {coef:12.4f} {se:12.4f} {tval:10.4f} {pval:14.10f} {ci_low:12.4f} {ci_high:12.4f}")
    print("==============================================================================")
    return m


def regress_hs_distance(ts, mask):
    """Hs ~ dist_km (+ dist_km²) within a mask (e.g., peak_mask)."""
    d = ts.loc[mask, ['hs','dist_km']].dropna()
    if len(d) < 5:
        print("[Hs~dist] Not enough rows:", len(d))
        return None
    X = pd.DataFrame({'dist_km': d['dist_km'], 'dist_km2': d['dist_km']**2})
    return sm.OLS(d['hs'], sm.add_constant(X)).fit()


def regress_exposure(ts, mask):
    """Hs ~ wind_ms + dist_km + pressure_anom within a mask."""
    cols = ['hs','wind_ms','pressure_anom','dist_km']
    d = ts.loc[mask, cols].dropna()
    if len(d) < 5:
        print("[Hs~wind+dist+press] Not enough rows:", len(d))
        return None
    return sm.OLS(d['hs'], sm.add_constant(d[['wind_ms','pressure_anom','dist_km']])).fit()


def single_var_reg(ts, xvar, mask):
    """Quick single-var scan Hs ~ xvar within a mask."""
    cols = ['hs', xvar]
    d = ts.loc[mask, cols].dropna()
    if len(d) < 10:
        return None
    m = sm.OLS(d['hs'], sm.add_constant(d[xvar])).fit()
    return xvar, m.rsquared, len(d), m.params.get(xvar, np.nan), m.pvalues.get(xvar, np.nan)


def oti_regression_suite(oti_like, label="OTI buoy", mask=None):
    """Run full regression suite on OTI (Hs ~ various forcings)."""
    if mask is None:
        mask = peak_mask

    print(f"\n=== {label}: regressions ({'peak_mask' if mask is peak_mask else 'custom mask'}) ===")

    # Multi-var models
    m1 = regress_hs_distance(oti_like, mask)
    m2 = regress_exposure(oti_like, mask)
    if m1 is not None:
        print(f"[{label}] Hs~dist: R2={m1.rsquared:.3f}, n={int(m1.nobs)}")
    if m2 is not None:
        print(f"[{label}] Hs~wind+dist+press: R2={m2.rsquared:.3f}, n={int(m2.nobs)}")

    # Explicit OLS tables (full precision p-values)
    d = oti_like.loc[mask, ['hs','dist_km','wind_ms','pressure_anom']].dropna()
    if len(d) >= 5:
        _ = _ols_report(f"{label} | Hs ~ dist",               d['hs'], d[['dist_km']])
        _ = _ols_report(f"{label} | Hs ~ wind",               d['hs'], d[['wind_ms']])
        _ = _ols_report(f"{label} | Hs ~ press",              d['hs'], d[['pressure_anom']])
        m_expo_oti = _ols_report(f"{label} | Hs ~ wind+dist+press", d['hs'], d[['wind_ms','dist_km','pressure_anom']])
        d_poly = d.assign(dist_km2=lambda x: x['dist_km']**2)
        _ = _ols_report(f"{label} | Hs ~ dist + dist²", d_poly['hs'], d_poly[['dist_km','dist_km2']])
    else:
        m_expo_oti = None
        print(f"[{label}] Not enough rows for printed OLS tables (n={len(d)}).")

    # Single-var scan
    results = [single_var_reg(oti_like, var, mask) for var in ['wind_ms','pressure_anom','dp']]
    results = [r for r in results if r]
    if results:
        print(f"\n{label} — Hs vs single forcings")
        for var, r2, n, coef, p in results:
            print(f"  {var:>14} | R²={r2:5.3f} | n={n:3d} | coef={coef:8.3f} | p={p:.10f}")

    # Energy flux summary (±3d)
    win = (oti_like.index >= (t_first - pad)) & (oti_like.index <= (t_last + pad))
    if 'energy_flux' in oti_like.columns:
        sub = oti_like.loc[win, 'energy_flux'].dropna()
        if len(sub):
            print(f"\n{label}: mean P={sub.mean():.1f}, max P={sub.max():.1f} (±3d)")
        else:
            print(f"\n{label}: no non-NaN energy_flux values in ±3d window.")
    else:
        print(f"\n{label}: 'energy_flux' column not present.")

    return {'m_dist': m1, 'm_exposure': m2, 'm_exposure_table': m_expo_oti}


# ---- Example execution (if running standalone) ----
_ = oti_regression_suite(oti if 'oti' in locals() else oti_frame, label="OTI buoy", mask=peak_mask)

## 9) Cross-correlation utilities

In [55]:
def best_cross_correlation(x, y, max_lag_hours=48, title=''):
    """Lag with max |corr|; positive = x leads y."""
    df = pd.concat([x.rename('x'), y.rename('y')], axis=1).dropna()
    if len(df) < 24:
        print(f"[{title}] Not enough data for cross-correlation (n={len(df)}).")
        return None
    xz = (df['x'] - df['x'].mean()) / df['x'].std(ddof=1)
    yz = (df['y'] - df['y'].mean()) / df['y'].std(ddof=1)
    lags = np.arange(-max_lag_hours, max_lag_hours+1, 1)
    cors = []
    for L in lags:
        if L < 0:  cors.append(np.corrcoef(xz[-L:].values, yz[:len(yz)+L].values)[0,1])
        elif L>0:  cors.append(np.corrcoef(xz[:len(xz)-L].values, yz[L:].values)[0,1])
        else:      cors.append(np.corrcoef(xz.values, yz.values)[0,1])
    best_idx = int(np.nanargmax(np.abs(cors)))
    best_lag = int(lags[best_idx]); best_corr = float(cors[best_idx])
    sign = "x leads y" if best_lag > 0 else ("y leads x" if best_lag < 0 else "simultaneous")
    print(f"\n{title}  best lag = {best_lag:+d} h ({sign}),  corr = {best_corr:.3f}")
    plt.figure(figsize=(6,3))
    plt.stem(lags, cors, use_line_collection=True)
    plt.xlabel('Lag (hours): positive = wind leads Hs'); plt.ylabel('Correlation'); plt.title(f'Cross-correlation: {title}')
    plt.tight_layout()
    return best_lag, best_corr

def crosscorr_plot(x, y, max_lag_hours=48, title='Cross-correlation'):
    """Plot only, same lag convention as above."""
    df = pd.concat([x.rename('x'), y.rename('y')], axis=1).dropna()
    if len(df) < 24:
        print(f"[{title}] Not enough data.")
        return
    xz = (df['x'] - df['x'].mean()) / df['x'].std(ddof=1)
    yz = (df['y'] - df['y'].mean()) / df['y'].std(ddof=1)
    lags = np.arange(-max_lag_hours, max_lag_hours+1, 1)
    cors = []
    for L in lags:
        if L < 0: cors.append(np.corrcoef(xz[-L:].values, yz[:len(yz)+L].values)[0,1])
        elif L>0: cors.append(np.corrcoef(xz[:len(xz)-L].values, yz[L:].values)[0,1])
        else:     cors.append(np.corrcoef(xz.values, yz.values)[0,1])
    best_idx = int(np.nanargmax(np.abs(cors))); best_lag = int(lags[best_idx]); best_corr = float(cors[best_idx])
    sign = "wind leads Hs" if best_lag > 0 else ("Hs leads wind" if best_lag < 0 else "simultaneous")
    print(f"{title}: best lag = {best_lag:+d} h ({sign}),  corr = {best_corr:.3f}")
    plt.figure(figsize=(6,3)); plt.stem(lags, cors)
    plt.xlabel('Lag (hours): positive = wind leads Hs'); plt.ylabel('Correlation'); plt.title(title); plt.tight_layout()

# Examples (keep your originals)
best_cross_correlation(oti.loc[win_mask, 'wind_ms'], oti.loc[win_mask, 'hs'],
                       max_lag_hours=48, title='OTI (wind_ms vs Hs, ±3d)')
for s, ts in sector_frames.items():
    best_cross_correlation(ts.loc[win_mask, 'wind_ms'], ts.loc[win_mask, 'hs'],
                           max_lag_hours=48, title=f'{s} (wind_ms vs Hs, ±3d)')

save_all_open_figs(prefix="storm_analysis")

C:\Users\arc\AppData\Local\Temp\ipykernel_38204\1609172813.py:20: MatplotlibDeprecationWarning: The 'use_line_collection' parameter of stem() was deprecated in Matplotlib 3.6 and will be removed two minor releases later. If any parameter follows 'use_line_collection', they should be passed as keyword, not positionally.
  plt.stem(lags, cors, use_line_collection=True)
C:\Users\arc\AppData\Local\Temp\ipykernel_38204\1609172813.py:20: MatplotlibDeprecationWarning: The 'use_line_collection' parameter of stem() was deprecated in Matplotlib 3.6 and will be removed two minor releases later. If any parameter follows 'use_line_collection', they should be passed as keyword, not positionally.
  plt.stem(lags, cors, use_line_collection=True)
C:\Users\arc\AppData\Local\Temp\ipykernel_38204\1609172813.py:20: MatplotlibDeprecationWarning: The 'use_line_collection' parameter of stem() was deprecated in Matplotlib 3.6 and will be removed two minor releases later. If any parameter follows 'use_line_coll

## 10) Visualisation

In [56]:
# --- imports needed for this block ---

def _save_current_fig(fname_stem: str):
    """Save the current matplotlib figure to PNG with tight layout."""
    out = FIGS_DIR / f"{fname_stem}.png"
    plt.tight_layout()
    plt.savefig(out, dpi=200, bbox_inches="tight")
    print("saved:", out.resolve())

# --- Single-sector exporters --------------------------------------------------
def export_sector_timeplot(sector, ts, mask=in_evt):
    title = f"{sector} sector: Hs vs Time (peak window)"
    plot_hs_time_with_reg(ts, title=title, mask=mask)
    _save_current_fig(f"{sector}_time_hs_peak")

def export_sector_wind_vs_hs(sector, ts, mask=in_evt):
    d = ts.loc[mask, ['wind_ms','hs']].dropna()
    if len(d) < 5:
        print(f"[{sector}] Not enough points for wind-vs-hs.")
        return
    x, y = d['wind_ms'].values, d['hs'].values
    b1, b0 = np.polyfit(x, y, 1)
    xr = np.linspace(x.min(), x.max(), 100)
    fig, ax = plt.subplots(figsize=(4.8,4))
    ax.scatter(x, y, s=12, alpha=0.8, label='Hourly points')
    ax.plot(xr, b1*xr + b0, lw=2, label=f'Fit: Hs = {b1:.3f}·wind + {b0:.3f}')
    r = np.corrcoef(x, y)[0,1]
    ax.set_xlabel('Wind speed (m/s)'); ax.set_ylabel('Hs (m)')
    ax.set_title(f"{sector} sector: Wind vs Hs (event)\nPearson r={r:.2f}")
    ax.legend()
    _save_current_fig(f"{sector}_scatter_wind_vs_hs_peak")

def export_sector_rose(sector, ts):
    directional_rose_pre_vs_peak(ts, f'{sector} sector')
    _save_current_fig(f"{sector}_rose_pre_vs_peak")

def export_sector_xcorr(sector, ts, mask=win_mask):
    crosscorr_plot(ts.loc[mask, 'wind_ms'], ts.loc[mask, 'hs'],
                   max_lag_hours=48, title=f'{sector} (wind vs Hs, ±3d)')
    _save_current_fig(f"{sector}_xcorr_wind_hs_pm3d")

# --- Multi-series exporters ---------------------------------------------------
def export_map_and_bars():
    map_reefs_cyclone_maxhs(sector_centroids, sector_frames, df_cyc)
    _save_current_fig("map_cyclone_track_and_sectors_maxHs")

    bar_energy_flux_by_sector(sector_frames, mask=in_evt, stat='mean')
    _save_current_fig("bar_energy_flux_mean_event")

# --- OTI buoy exporters -------------------------------------------------------
def export_oti_all():
    plot_hs_time_with_reg(oti, title='OTI buoy: Hs vs Time (peak window)', mask=in_evt)
    _save_current_fig("OTI_time_hs_peak")

    d = oti.loc[in_evt, ['wind_ms','hs']].dropna()
    if len(d) >= 5:
        x, y = d['wind_ms'].values, d['hs'].values
        b1, b0 = np.polyfit(x, y, 1)
        xr = np.linspace(x.min(), x.max(), 100)
        fig, ax = plt.subplots(figsize=(4.8,4))
        ax.scatter(x, y, s=12, alpha=0.8, label='Hourly points')
        ax.plot(xr, b1*xr + b0, lw=2, label=f'Fit: Hs = {b1:.3f}·wind + {b0:.3f}')
        r = np.corrcoef(x, y)[0,1]
        ax.set_xlabel('Wind speed (m/s)'); ax.set_ylabel('Hs (m)')
        ax.set_title('OTI buoy: Wind vs Hs (event)\nPearson r={:.2f}'.format(r))
        ax.legend()
        _save_current_fig("OTI_scatter_wind_vs_hs_peak")

    directional_rose_pre_vs_peak(oti, 'OTI buoy')
    _save_current_fig("OTI_rose_pre_vs_peak")

    crosscorr_plot(oti.loc[win_mask, 'wind_ms'], oti.loc[win_mask, 'hs'],
                   max_lag_hours=48, title='OTI (wind vs Hs, ±3d)')
    _save_current_fig("OTI_xcorr_wind_hs_pm3d")

# --- Batch: regenerate everything with corrected labels ----------------------
def export_all_figs():
    export_map_and_bars()
    export_oti_all()
    for sector, ts in sector_frames.items():
        export_sector_timeplot(sector, ts, mask=in_evt)
        export_sector_wind_vs_hs(sector, ts, mask=in_evt)
        export_sector_rose(sector, ts)
        export_sector_xcorr(sector, ts, mask=win_mask)

# Run once to regenerate with swapped North/South labels:
# export_all_figs()

# --- Plotting helpers (unchanged except imports/typo fix) --------------------
def plot_hs_time_with_reg(series, title='Hs vs Time (with linear trend)', mask=None):
    """Line plot of Hs with simple linear fit over selected window."""
    d = series[['hs']].copy()
    if mask is not None:
        d = d.loc[mask]
    d = d.dropna()
    if d.empty:
        print(f"[{title}] No data.")
        return
    t0_ = d.index[0]
    x = (d.index - t0_).total_seconds() / 3600.0
    y = d['hs'].values
    X = np.vstack([np.ones_like(x), x]).T
    coef = np.linalg.lstsq(X, y, rcond=None)[0]
    yhat = X @ coef
    fig, ax = plt.subplots(figsize=(9,3))
    ax.plot(d.index, y, lw=1, label='Hs')
    ax.plot(d.index, yhat, lw=2, label='Linear trend')
    ax.set_title(title); ax.set_ylabel('Hs (m)')
    ax.xaxis.set_major_formatter(DateFormatter('%m-%d\n%H:%M'))
    ax.legend(); plt.tight_layout()

def map_reefs_cyclone_maxhs(sector_centroids, sector_frames, df_cyc, oti_lat=-23.504, oti_lon=152.094):
    """Simple lon/lat map: cyclone track + sector centroid with max Hs label."""
    rows = []
    for _, r in sector_centroids.iterrows():
        name = r['sector']; ts = sector_frames.get(name)
        if ts is None or 'hs' not in ts:
            continue
        hs_max = ts.loc[in_evt, 'hs'].max()
        rows.append((name, float(r['sector_lat']), float(r['sector_lon']), hs_max))
    df_max = pd.DataFrame(rows, columns=['sector','lat','lon','hs_max'])
    fig, ax = plt.subplots(figsize=(6,6))
    ax.plot(df_cyc['cyc_lon'], df_cyc['cyc_lat'], lw=1.5, label='Cyclone track')
    ax.scatter(df_max['lon'], df_max['lat'], s=80, label='Reef sectors')
    for _, r in df_max.iterrows():
        ax.text(r['lon']+0.02, r['lat']+0.02, f"{r['sector']} (max Hs={r['hs_max']:.1f} m)", fontsize=9)
    ax.scatter([oti_lon],[oti_lat], marker='^', s=100, label='OTI buoy')
    ax.text(oti_lon+0.02, oti_lat+0.02, "OTI", fontsize=9)
    all_lons = np.r_[df_max['lon'].values, df_cyc['cyc_lon'].values, oti_lon]
    all_lats = np.r_[df_max['lat'].values, df_cyc['cyc_lat'].values, oti_lat]
    ax.set_xlim(all_lons.min()-0.3, all_lons.max()+0.3)
    ax.set_ylim(all_lats.min()-0.3, all_lats.max()+0.3)
    ax.set_xlabel('Longitude'); ax.set_ylabel('Latitude')
    ax.set_title('Cyclone track + sectors (max Hs during event)')
    ax.legend(); ax.grid(True, ls=':'); plt.tight_layout()

def bar_energy_flux_by_sector(sector_frames, mask=in_evt, stat='mean'):
    """Bar chart of sector energy flux (mean or max) over mask."""
    vals = []
    for s, ts in sector_frames.items():
        if 'energy_flux' not in ts:
            continue
        v = ts.loc[mask, 'energy_flux'].dropna()
        if v.empty:
            continue
        vals.append((s, v.max() if stat == 'max' else v.mean()))
    dfp = pd.DataFrame(vals, columns=['sector','P']).sort_values('P', ascending=False)
    fig, ax = plt.subplots(figsize=(6,3))
    ax.bar(dfp['sector'], dfp['P'])
    ax.set_ylabel('Energy flux P (W/m)')
    ax.set_title(f'Energy flux per sector ({stat} during event)')
    plt.tight_layout()
    return dfp

def rose_plot(ax, angles_deg, bins=16, title=''):
    """Polar histogram of directions."""
    a = np.asarray(angles_deg, dtype=float)
    a = a[~np.isnan(a)]
    if len(a) == 0:
        ax.set_title(title + ' (no data)')
        return
    theta = np.deg2rad(a % 360.0)
    edges = np.linspace(0, 2*np.pi, bins+1)
    hist, _ = np.histogram(theta, bins=edges)
    widths = np.diff(edges)
    ax.bar(edges[:-1], hist, width=widths, align='edge', edgecolor='k', linewidth=0.5)
    ax.set_title(title)

def directional_rose_pre_vs_peak(df, name='Series'):
    """Two-panel rose plot: pre vs peak."""
    pre  = df.loc[pre_evt,  'dp'].dropna() if 'dp' in df else pd.Series(dtype=float)
    peak = df.loc[in_evt,   'dp'].dropna() if 'dp' in df else pd.Series(dtype=float)
    fig = plt.figure(figsize=(8,4))
    ax1 = plt.subplot(1,2,1, projection='polar'); rose_plot(ax1, pre.values,  bins=16, title=f'{name}: pre')
    ax2 = plt.subplot(1,2,2, projection='polar'); rose_plot(ax2, peak.values, bins=16, title=f'{name}: peak')
    plt.tight_layout()

# --- Example visual calls (unchanged) ----------------------------------------
plot_hs_time_with_reg(oti,  title='OTI buoy: Hs vs Time (peak window)', mask=in_evt)
for s, ts in sector_frames.items():
    plot_hs_time_with_reg(ts, title=f'{s} sector: Hs vs Time (peak window)', mask=in_evt)
map_reefs_cyclone_maxhs(sector_centroids, sector_frames, df_cyc)
dfp_mean = bar_energy_flux_by_sector(sector_frames, mask=in_evt, stat='mean')
directional_rose_pre_vs_peak(oti, 'OTI buoy')
for s, ts in sector_frames.items():
    directional_rose_pre_vs_peak(ts, f'{s} sector')
crosscorr_plot(oti.loc[win_mask, 'wind_ms'], oti.loc[win_mask, 'hs'],
               max_lag_hours=48, title='OTI (wind vs Hs, ±3d)')
for s, ts in sector_frames.items():
    crosscorr_plot(ts.loc[win_mask, 'wind_ms'], ts.loc[win_mask, 'hs'],
                   max_lag_hours=48, title=f'{s} (wind vs Hs, ±3d)')
save_all_open_figs(prefix="visualisations")

## 11) Model diagnostics

In [57]:
def model_fit_diagnostics(df, ycol='hs', xcols=('wind_ms','dist_km','pressure_anom'),
                          mask=None, title='Model diagnostics'):
    """OLS fit + R²/DW/Ljung–Box + residual plots."""
    d = df.loc[mask, [ycol, *xcols]].dropna()
    if len(d) < (len(xcols) + 5):
        print(f"[{title}] Not enough rows (n={len(d)})")
        return None
    X = sm.add_constant(d[list(xcols)], has_constant='add')
    y = d[ycol]
    m = sm.OLS(y, X).fit()
    res = m.resid

    print(f"\n[{title}] R²={m.rsquared:.3f} | adj.R²={m.rsquared_adj:.3f} | n={int(m.nobs)}")
    print(f"  Durbin–Watson: {sm.stats.durbin_watson(res):.2f}")
    for lag in (6,12,24):
        lb = acorr_ljungbox(res, lags=[lag], return_df=True)['lb_pvalue'].iloc[0]
        print(f"  Ljung–Box lag {lag}: p={lb:.3f}")

    fig = plt.figure(figsize=(12,3.2))
    plt.subplot(1,3,1); plt.scatter(m.fittedvalues, res, s=10); plt.axhline(0, ls='--', lw=1)
    plt.xlabel('Fitted'); plt.ylabel('Residual'); plt.title('Residuals vs Fitted')
    plt.subplot(1,3,2); sm.qqplot(res, line='45', fit=True, ax=plt.gca()); plt.title('QQ plot')
    plt.subplot(1,3,3); sm.graphics.tsa.plot_acf(res, lags=36, ax=plt.gca()); plt.title('Residual ACF (36 lags)')
    fig.suptitle(title, y=1.05, fontsize=11); plt.tight_layout()
    return m

# Diagnostics for OTI buoy + sectors (peak)
m_oti = model_fit_diagnostics(oti, xcols=('wind_ms','dist_km','pressure_anom'),
                              mask=in_evt, title='OTI buoy: Hs ~ wind+dist+press (peak)')
models_sector = {s: model_fit_diagnostics(ts, xcols=('wind_ms','dist_km','pressure_anom'),
                                          mask=in_evt, title=f'{s} sector: Hs ~ wind+dist+press (peak)')
                 for s, ts in sector_frames.items()}

save_all_open_figs(prefix="Diagnostics")

## 12) Statistical tests (energy flux & circular ANOVA)

In [58]:
# --- Imports (deduped) -------------------------------------------------------
import numpy as np
import pandas as pd
from itertools import combinations
from scipy import stats

# --- Energy flux: summary + pairwise tests -----------------------------------
def energy_flux_tests_all(sector_frames, mask, sectors=('North', 'Central', 'South')):
    """
    Summarize energy_flux per sector over `mask`, then run pairwise tests:
      - Welch t-test (unequal variances)
      - Mann–Whitney U (two-sided)
    """
    # Pull series and basic diagnostics
    P = {}
    for s in sectors:
        if s not in sector_frames:
            print(f"[warn] sector '{s}' not found in sector_frames.")
            continue
        ts = sector_frames[s]
        if 'energy_flux' not in ts.columns:
            print(f"[warn] sector '{s}' has no 'energy_flux' column.")
            P[s] = pd.Series(dtype=float)
            continue
        P[s] = ts.loc[mask, 'energy_flux'].dropna()

    # Summary table
    print("\nEnergy flux samples per sector:")
    for s in sectors:
        v = P.get(s, pd.Series(dtype=float))
        print(f"  {s:7s}  n={len(v):4d}  mean={v.mean():.1f}  sd={v.std(ddof=1):.1f}")

    # Guard: need at least 2 sectors with ≥3 samples
    valid = [s for s in sectors if len(P.get(s, [])) >= 3]
    if len(valid) < 2:
        print("\nNot enough data for pairwise tests (need ≥3 pts in at least two sectors).")
        return

    # Pairwise tests
    print("\nPairwise tests (Welch t-test | Mann–Whitney U):")
    for a, b in combinations(valid, 2):
        va, vb = P[a], P[b]
        t_w, p_w = stats.ttest_ind(va, vb, equal_var=False, nan_policy='omit')
        u, p_u = stats.mannwhitneyu(va, vb, alternative='two-sided')
        print(f"  {a:7s} vs {b:7s}  t={t_w:7.3f}, p={p_w:.4g}   |   U={u:7.0f}, p={p_u:.4g}")

def energy_flux_samples_oti(mask=in_evt, label="OTI"):
    """
    Print OTI energy-flux sample size, mean, and SD over `mask`,
    matching the sector summary format.
    """
    # pick the right column (you computed P_buoy earlier)
    frame = oti if 'oti' in globals() else oti_frame
    col = 'energy_flux' if 'energy_flux' in frame.columns else ('P_buoy' if 'P_buoy' in frame.columns else None)
    if col is None:
        print(f"{label}: no 'energy_flux' or 'P_buoy' column found.")
        return

    v = frame.loc[mask, col].dropna()
    print("\nEnergy flux samples (OTI):")
    print(f"  {label:7s} n={len(v):4d}  mean={v.mean():.1f}  sd={v.std(ddof=1):.1f}")


# --- Watson–Williams (circular ANOVA) implementation (degrees) ---------------
def _estimate_kappa(Rbar):
    """Approximate kappa from mean resultant length Rbar (Zar/Berens)."""
    if Rbar < 0.53:
        return 2*Rbar + Rbar**3 + (5*Rbar**5)/6
    elif Rbar < 0.85:
        return -0.4 + 1.39*Rbar + 0.43/(1 - Rbar)
    else:
        # avoid div by zero
        eps = 1e-12
        return 1.0 / max(Rbar**3 - 4*Rbar**2 + 3*Rbar, eps)

def watson_williams_test(data_dict):
    """
    Watson–Williams test for equality of circular means across groups.
    data_dict: {'group': array_like of angles in degrees, ...}
    Returns dict: F, df1, df2, p, per-group n and mean directions (deg).
    """
    # Clean groups: wrap to [0, 360), drop NaNs, require n>=2
    groups = {}
    for g, a in data_dict.items():
        if a is None:
            continue
        x = np.asarray(a, dtype=float)
        x = x[np.isfinite(x)]
        if x.size >= 2:
            groups[g] = np.mod(x, 360.0)

    k = len(groups)
    if k < 2:
        raise ValueError("Need at least two groups with >=2 observations each.")

    # Convert to radians
    rad = {g: np.deg2rad(v) for g, v in groups.items()}

    # Per-group sums
    n_i = {g: v.size for g, v in rad.items()}
    C_i = {g: np.sum(np.cos(v)) for g, v in rad.items()}
    S_i = {g: np.sum(np.sin(v)) for g, v in rad.items()}
    R_i = {g: np.hypot(C_i[g], S_i[g]) for g in groups}

    # Pooled
    N = np.sum(list(n_i.values()))
    C = np.sum(list(C_i.values()))
    S = np.sum(list(S_i.values()))
    R = np.hypot(C, S)

    # Concentration estimate and small-sample correction
    Rbar = R / N
    kappa = _estimate_kappa(Rbar)
    c = 1.0 + 3.0/(8.0 * max(kappa, 1e-12))  # small-sample correction

    # WW F-statistic (Berens' CircStat style)
    df1 = k - 1
    df2 = N - k
    num = (N - k) * (np.sum(list(R_i.values())) - R)
    den = (k - 1) * (N - np.sum(list(R_i.values())))
    if den <= 0:
        F = 0.0
        p = 1.0
    else:
        F = c * (num / den)
        F = float(max(F, 0.0))  # clamp tiny negatives
        p = stats.f.sf(F, df1, df2)

    # Group circular means (deg)
    means_deg = {}
    for g in groups:
        mu = np.degrees(np.arctan2(S_i[g]/n_i[g], C_i[g]/n_i[g])) % 360.0
        means_deg[g] = mu

    return {
        'F': F, 'df1': df1, 'df2': df2, 'p': p,
        'n': n_i, 'means_deg': means_deg, 'kappa_hat': kappa, 'Rbar': Rbar
    }

# --- Wrapper: use Pingouin if available, else built-in WW --------------------
def circ_anova_by_group(data_dict):
    """
    Try Pingouin's `watson_williams` if present; else use the built-in
    Watson–Williams implementation above. Prints a compact summary.
    """
    # Tidy: drop empty groups early
    cleaned = {g: pd.Series(a, dtype=float).dropna().values for g, a in data_dict.items()}
    cleaned = {g: a for g, a in cleaned.items() if a.size >= 2}
    if len(cleaned) < 2:
        print("No directional data available for circular ANOVA (need ≥2 groups with ≥2 obs).")
        return None

    # Prefer Pingouin if available in this environment
    try:
        import pingouin as pg
        if hasattr(pg, 'watson_williams'):
            df = pd.DataFrame(
                [{'dp': float(v), 'group': g}
                 for g, arr in cleaned.items()
                 for v in np.mod(arr, 360.0)]
            )
            res = pg.watson_williams(dv='dp', between='group', data=df)
            print("\nCircular ANOVA (Watson–Williams) via Pingouin:")
            print(res)
            return res
    except Exception:
        # fall through to built-in
        pass

    # Built-in implementation
    ww = watson_williams_test(cleaned)
    print("\nCircular ANOVA (Watson–Williams) – built-in implementation")
    print(f"F({ww['df1']}, {ww['df2']}) = {ww['F']:.3f},  p = {ww['p']:.4g}")
    print("Group means (deg):", {k: round(v, 1) for k, v in ww['means_deg'].items()})
    return ww

# --- Convenience: run by sector (peak) + by phase for any series -------------
# Example call for sectors (peak window):
groups_peak = {s: ts.loc[in_evt, 'dp'].values for s, ts in sector_frames.items() if 'dp' in ts}
circ_anova_by_group(groups_peak)

def anova_by_phase(ts, name='Series'):
    """Watson–Williams on dp across phases (pre/peak/post) for a single series."""
    ph = {}
    if 'dp' in ts:
        ph['pre']  = ts.loc[pre_evt,  'dp'].values
        ph['peak'] = ts.loc[in_evt,   'dp'].values
        ph['post'] = ts.loc[post_evt, 'dp'].values
    print(f"\n{name} – Circular ANOVA of dp by phase (pre/peak/post):")
    return circ_anova_by_group(ph)

# Example calls:
anova_by_phase(sector_frames['North'], 'North')
anova_by_phase(sector_frames['Central'], 'Central')
anova_by_phase(sector_frames['South'], 'South')
anova_by_phase(oti, 'OTI buoy')

{'F': 6.365450602179527,
 'df1': 2,
 'df2': 1208,
 'p': 0.001778220325484958,
 'n': {'pre': 480, 'peak': 379, 'post': 352},
 'means_deg': {'pre': 52.34966826527471,
  'peak': 48.44442269056289,
  'post': 51.573152081202494},
 'kappa_hat': 12.51315110882625,
 'Rbar': 0.959174663300831}

## 13) Quick sanity prints

In [59]:
print("\nWindows:")
print("  OTI (buoy) idx:", oti.index.min(), "→", oti.index.max(), "| n:", len(oti))
print("  Cyclone peak: ", t_first, "→", t_last)
print("  Regr rows (peak):", len(df_reg))
print("dims:", ds_wave.dims)
print("coords:", list(ds_wave.coords))
print("data vars (first 10):", list(ds_wave.data_vars)[:10])
try:
    print("lat shape:", ds_wave['lat'].shape)
    print("lon shape:", ds_wave['lon'].shape)
except Exception as e:
    print("No 'lat'/'lon' variable:", e)
print("Raw buoy headers:", list(df_buoy.columns)[:20])
print(df_buoy.head(3))

## 14) Save Printed Outputs

In [60]:
from contextlib import redirect_stdout, redirect_stderr
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import statsmodels.api as sm

# ---- Log destination
LOG_DIR = Path("logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)

# ---- Helpers to access OTI and make sure energy flux exists
def _get_oti_like():
    """Return (DataFrame, label) for the OTI buoy table present in globals()."""
    if 'oti' in globals():
        return oti, "OTI buoy"
    if 'oti_frame' in globals():
        return oti_frame, "OTI buoy"
    raise NameError("Neither `oti` nor `oti_frame` is defined.")

def energy_flux_samples_oti(mask, label="OTI"):
    """Print OTI energy-flux sample size, mean, and SD over mask (matches sector format)."""
    frame, _ = _get_oti_like()
    col = 'energy_flux' if 'energy_flux' in frame.columns else ('P_buoy' if 'P_buoy' in frame.columns else None)
    if col is None:
        print(f"{label}: no 'energy_flux' or 'P_buoy' column found.")
        return
    v = frame.loc[mask, col].dropna()
    print("\nEnergy flux samples (OTI):")
    print(f"  {label:7s} n={len(v):4d}  mean={v.mean():.1f}  sd={v.std(ddof=1):.1f}")

# Ensure OTI has an 'energy_flux' column if only P_buoy is present
try:
    _oti_df, _ = _get_oti_like()
    if 'energy_flux' not in _oti_df.columns and 'P_buoy' in _oti_df.columns:
        _oti_df = _oti_df.copy()
        _oti_df['energy_flux'] = _oti_df['P_buoy']
        if 'oti' in globals():
            oti = _oti_df
        else:
            oti_frame = _oti_df
except Exception:
    pass  # if OTI not present yet, skip

# ---- Directional shift summaries
def _print_dirshift_sectors():
    print("\n=== Directional Shift Analysis (per sector) ===")
    rows_dir = []
    for sector, ts in sector_frames.items():
        pre  = ts.loc[pre_evt,  'dp'].dropna() if 'dp' in ts else pd.Series(dtype=float)
        peak = ts.loc[in_evt,   'dp'].dropna() if 'dp' in ts else pd.Series(dtype=float)
        post = ts.loc[post_evt, 'dp'].dropna() if 'dp' in ts else pd.Series(dtype=float)
        pre_m, peak_m, post_m = circ_mean_deg(pre), circ_mean_deg(peak), circ_mean_deg(post)
        d_pre_peak  = circ_diff(pre_m,  peak_m)
        d_peak_post = circ_diff(peak_m, post_m)
        d_pre_post  = circ_diff(pre_m,  post_m)
        print(f"{sector:8s}  mean°  pre:{pre_m:6.1f}  peak:{peak_m:6.1f}  post:{post_m:6.1f}  "
              f"Δ(pre→peak):{d_pre_peak:6.1f}  Δ(peak→post):{d_peak_post:6.1f}  Δ(pre→post):{d_pre_post:6.1f}")
        rows_dir.append({'sector': sector, 'pre_mean': pre_m, 'peak_mean': peak_m, 'post_mean': post_m,
                         'Δ_pre_peak': d_pre_peak, 'Δ_peak_post': d_peak_post, 'Δ_pre_post': d_pre_post})
    df_dirshift = pd.DataFrame(rows_dir)
    if not df_dirshift.empty:
        print("\nSector directional shift summary (rounded):")
        with pd.option_context('display.max_columns', None):
            print(df_dirshift.round(1))

def _print_dirshift_oti():
    print("\n=== Directional Shift Analysis (OTI buoy) ===")
    oti_like, label = _get_oti_like()
    if 'dp' not in oti_like.columns:
        print(f"[{label}] No 'dp' column found — skipping.")
        return
    pre_dp  = oti_like.loc[pre_mask,  'dp'].dropna()
    peak_dp = oti_like.loc[peak_mask, 'dp'].dropna()
    post_dp = oti_like.loc[post_mask, 'dp'].dropna()
    pre_m  = circ_mean_deg(pre_dp.values)
    peak_m = circ_mean_deg(peak_dp.values)
    post_m = circ_mean_deg(post_dp.values)
    print(f"{label:8s} mean°  pre:{pre_m:6.1f}  peak:{peak_m:6.1f}  post:{post_m:6.1f}  "
          f"Δ(pre→peak):{circ_diff(pre_m,peak_m):6.1f}  Δ(peak→post):{circ_diff(peak_m,post_m):6.1f}  "
          f"Δ(pre→post):{circ_diff(pre_m,post_m):6.1f}")
    print(f"           n      pre:{len(pre_dp):6d}  peak:{len(peak_dp):6d}  post:{len(post_dp):6d}")

# ---- Regressions (sector + OTI)
def single_var_reg(ts, xvar, mask=None):
    """Quick single-var regression: Hs ~ xvar within mask (defaults to in_evt)."""
    if mask is None:
        mask = in_evt
    if xvar not in ts.columns or 'hs' not in ts.columns:
        return None
    d = ts.loc[mask, ['hs', xvar]].copy()
    d[xvar] = pd.to_numeric(d[xvar], errors='coerce')
    d['hs'] = pd.to_numeric(d['hs'], errors='coerce')
    d = d.dropna()
    if len(d) < 10:
        return None
    X = sm.add_constant(d[xvar], has_constant='add')
    m = sm.OLS(d['hs'], X).fit()
    return xvar, m.rsquared, int(m.nobs), float(m.params.get(xvar, np.nan)), float(m.pvalues.get(xvar, np.nan))

def _print_sector_regressions():
    print("\n=== Sector regressions ===")
    for sector, ts in sector_frames.items():
        try:
            m1 = regress_hs_distance(ts, mask=in_evt)          # uses your defined function
            m2 = regress_exposure(ts,     mask=in_evt)          # uses your defined function
            print(f"[{sector}] Hs~dist: R2={m1.rsquared:.3f}, n={int(m1.nobs)}")
            print(f"[{sector}] Hs~wind+dist+press: R2={m2.rsquared:.3f}, n={int(m2.nobs)}")
        except Exception as e:
            print(f"[{sector}] regression error:", repr(e))
        results = []
        for var in ['wind_ms', 'pressure_anom', 'dp']:
            out = single_var_reg(ts, var, mask=in_evt)
            if out: results.append(out)
        if results:
            print(f"{sector} — Hs vs single forcings (cyclone window)")
            for var, r2, n, coef, p in results:
                print(f"  {var:>14} | R²={r2:5.3f} | n={n:3d} | coef={coef:8.3f} | p={p:8.3e}")

def _print_oti_regressions():
    print("\n=== OTI regressions ===")
    oti_like, label = _get_oti_like()
    _ = oti_regression_suite(oti_like, label=label, mask=peak_mask)  # your existing suite

# =========================
# Main report writer
# =========================
def run_text_reports_to_file(filepath: Path):
    """
    Run all text-only reports and write every print to `filepath`.
    Captures both stdout and stderr so warnings from statsmodels are included.
    """
    with open(filepath, "w", encoding="utf-8") as f, \
         redirect_stdout(f), \
         redirect_stderr(f), \
         pd.option_context('display.width', 140, 'display.max_columns', None, 'display.max_rows', 500):

        print("=== Cyclone/Wave Analysis Report ===")
        print("Log file:", filepath)
        print("Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

        # Context line: storm + window (t_first/t_last from your cyclone window)
        try:
            print(f"Storm: {STORM} | Window: {t_first:%Y-%m-%d %H:%M} → {t_last:%Y-%m-%d %H:%M}")
        except Exception:
            print("(Storm metadata unavailable)")

        # Energy flux (sectors + OTI one-liner in the same format)
        print("\n-- Energy flux tests (pairwise) --")
        energy_flux_tests_all(sector_frames, mask=in_evt)
        energy_flux_samples_oti(mask=in_evt, label="OTI")

        # Circular ANOVA by sector (peak window)
        print("\n-- Circular ANOVA by sector (peak) --")
        groups_peak = {s: ts.loc[in_evt, 'dp'].values for s, ts in sector_frames.items() if 'dp' in ts}
        circ_anova_by_group(groups_peak)

        # Circular ANOVA by phase (North/Central/South + OTI) — single pass, no duplicates
        print("\n-- Circular ANOVA by phase --")
        for sname in ['North', 'Central', 'South']:
            anova_by_phase(sector_frames[sname], sname)
        oti_like, label = _get_oti_like()
        anova_by_phase(oti_like, label)

        # Directional shift summaries
        print("\n-- Directional shift summaries --")
        _print_dirshift_sectors()
        _print_dirshift_oti()

        # Regressions (sectors + OTI)
        print("\n-- Regressions --")
        _print_sector_regressions()
        _print_oti_regressions()

        print("\n=== END OF REPORT ===")

# =========================
# Execute once to generate a log
# =========================
log_name = f"analysis_log_{datetime.now():%Y%m%d_%H%M%S}.txt"
log_path = LOG_DIR / log_name
run_text_reports_to_file(log_path)
print("Saved log to:", log_path.resolve())
